In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sb
import pandas as pd
import os
import ipywidgets as widgets
import warnings
warnings.filterwarnings("ignore")
from turtle import color
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
 
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from Project.Utils.visualize import  get_zone
from sklearn.neighbors import KNeighborsRegressor
from Project.Utils.visualize import normalize_by_country


In [2]:
output_path = os.getcwd() + '/Output/'
cluster_path = output_path + '/Cluster/'
df_gold = pd.read_csv(output_path + 'GoldDataframe.csv', index_col = ['Country', 'Year', 'Region'])
df_cluster = pd.read_csv(cluster_path + 'All indicators.csv')
country_list = list(np.sort(df_gold.index.get_level_values('Country').unique()))

In [3]:
def get_cluster_countries(country):
    #Given a country, obatin data from the same countries in the cluster. 
    country_cluster_target = df_cluster.loc[df_cluster['Country'] == country]['Cluster'].item()
    country_cluster_list = df_cluster.loc[df_cluster['Cluster'] == country_cluster_target]['Country'].tolist()

    df = df_gold.loc[df_gold.index.get_level_values('Country').isin(country_cluster_list)].sort_index(level = 1)
    df = normalize_by_country(df)
    df.dropna(axis=1, inplace=True)
    df.reset_index(drop = True, inplace=True)
    
    return df, country_cluster_list

In [4]:
def machine_algorithm(df, country_cluster_list):
      #Separate Features and target
      X_Target= df.iloc[len(country_cluster_list):]  ##Original gdp
      X_Target = X_Target.loc[:, X_Target.columns == "GDP"] 
      feature_df= df.iloc[:-len(country_cluster_list)]  ##Indicators
      feature_df= feature_df.loc[:, feature_df.columns != 'GDP']
      #Split the data
      X_train, X_test, y_train, y_test = train_test_split(feature_df, X_Target, test_size=0.3, random_state=0)
      alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]
      elastic_cv= ElasticNetCV(alphas=alphas, cv=5, max_iter = 100000, tol = 0.001)
      model = elastic_cv.fit(X_train, y_train)
      ypred = model.predict(X_test)
      score = model.score(X_test, y_test)
      mse = mean_squared_error(y_test, ypred)
      #print("R2:{0:.4f}, MSE:{1:.4f}, RMSE:{2:.4f}"
      #      .format(score, mse, np.sqrt(mse)))
      return model, score, feature_df

In [5]:
def plot_machine(model, score, feature_df, country):
    df_country = df_gold.loc[df_gold.index.get_level_values('Country') == country]
    df_country = normalize_by_country(df_country)

    pred_df  = df_country.copy().dropna(axis=1)
    pred_df= pred_df.loc[:, pred_df.columns != 'GDP']
    pred_df= pred_df.loc[:, pred_df.columns.isin(feature_df.columns)] # You have to use the same indicators as the trained model (all countries must have data for the indicator, if not is dropped)
    prediction  = model.predict(pred_df)

    print(prediction)
    aux_line = np.array([df_country['GDP'][-1],prediction[-1]])
    std_1 = 1 - score

    plt.plot(2021, prediction[-1], 'ro', label='Prediction') ##Dot
    plt.fill_between(np.array([2020,2021]), aux_line * (1 - std_1), aux_line * (1 + std_1), color='r', alpha=0.2) ##Fill
    plt.plot([2020,2021], aux_line, 'b-', color = 'red', linestyle='--') ##Discontinous line
    plt.plot(df_country.index.get_level_values('Year').values, df_country['GDP'], 'b-', label='Original', color = 'blue') ##Original GDP

    plt.legend()
    plt.show()


In [6]:
def table_machine(zone):
    df, country_cluster_list = get_cluster_countries(zone)
    model, score, feature_df = machine_algorithm(df, country_cluster_list)
    plot_machine(model, score, feature_df, zone)


zone_drop_machine = widgets.Dropdown(
    options= country_list,
    value ='Afghanistan',
    description='Zone:',
)

widgets.interact(table_machine, zone = zone_drop_machine)

interactive(children=(Dropdown(description='Zone:', options=('Afghanistan', 'Albania', 'Algeria', 'Angola', 'A…

<function __main__.table_machine(zone)>

In [7]:
df = df_gold.loc[df_gold.index.get_level_values('Country') == 'Spain']
df = normalize_by_country(df)
df

AgriShareGDP  CreditToAgriFishForest  \
Country Year Region                                                          
Spain   2000 Europe and Central Asia      1.000000                0.375098   
        2001 Europe and Central Asia      1.000000                0.263661   
        2002 Europe and Central Asia      0.866434                0.399710   
        2003 Europe and Central Asia      0.825463                0.372793   
        2004 Europe and Central Asia      0.611023                0.436677   
        2005 Europe and Central Asia      0.356609                0.506828   
        2006 Europe and Central Asia      0.103548                0.474867   
        2007 Europe and Central Asia      0.171719                0.216057   
        2008 Europe and Central Asia      0.072943                0.251587   
        2009 Europe and Central Asia      0.000000                0.138120   
        2010 Europe and Central Asia      0.117321                0.000000   
        2011 Europe and Central Asia      0.076349                0.010524   
        2012 Europe and Central Asia      0.101155                0.122189   
        2013 Europe and Central Asia      0.252721                0.065741   
        2014 Europe and Central Asia      0.191823                0.161290   
        2015 Europe and Central Asia      0.332753                0.148679   
        2016 Europe and Central Asia      0.396738                0.288116   
        2017 Europe and Central Asia      0.370403                0.494199   
        2018 Europe and Central Asia      0.351022                0.791029   
        2019 Europe and Central Asia      0.247481                1.000000   
        2020 Europe and Central Asia      0.624965                1.000000   

                                      EmploymentRural       GDP  % Soldiers  \
Country Year Region                                                           
Spain   2000 Europe and Central Asia         0.670979  0.000000    1.000000   
        2001 Europe and Central Asia         0.720201  0.029564    0.695104   
        2002 Europe and Central Asia         0.776438  0.105551    0.989051   
        2003 Europe and Central Asia         0.820794  0.300056    0.589716   
        2004 Europe and Central Asia         0.872617  0.457272    0.469992   
        2005 Europe and Central Asia         0.923919  0.541011    0.387588   
        2006 Europe and Central Asia         0.941679  0.644105    0.345115   
        2007 Europe and Central Asia         0.994741  0.851003    0.291997   
        2008 Europe and Central Asia         1.000000  1.000000    0.245681   
        2009 Europe and Central Asia         0.869895  0.864070    0.217347   
        2010 Europe and Central Asia         0.829630  0.801088    0.207460   
        2011 Europe and Central Asia         0.809825  0.857535    0.136761   
        2012 Europe and Central Asia         0.729579  0.707835    0.134968   
        2013 Europe and Central Asia         0.671964  0.736945    0.136401   
        2014 Europe and Central Asia         0.686114  0.751182    0.026025   
        2015 Europe and Central Asia         0.768035  0.581717    0.030749   
        2016 Europe and Central Asia         0.854399  0.617653    0.013496   
        2017 Europe and Central Asia         0.000000  0.695893    0.000000   
        2018 Europe and Central Asia         0.031012  0.801352    0.001715   
        2019 Europe and Central Asia         0.051402  0.774176    0.001715   
        2020 Europe and Central Asia         0.026925  0.665696    0.001715   

                                      Employment in industry  \
Country Year Region                                            
Spain   2000 Europe and Central Asia                0.947544   
        2001 Europe and Central Asia                1.000000   
        2002 Europe and Central Asia                0.965029   
        2003 Europe and Central Asia                0.937552   
        2004 Europe and Central Asia     